In [22]:
import ee
# import geemap/
import geemap.foliumap as geemap


ee.Authenticate()
ee.Initialize()
# for 2024 
table = ee.FeatureCollection("users/banditmienmany/trainingsample")
ee_object = geemap.shp_to_ee("study_area.shp")

# shapefile = ee.FeatureCollection('users/your_username/asset_name')

# To use the shapefile's geometry, you might need to extract the geometry.
# If the shapefile contains multiple features, you may need to merge them into a single geometry.
# Here's an example of how to merge geometries if there are multiple features.
geometry = ee_object.geometry()
# Function to mask clouds using the Sentinel-2 QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Image collection for 2024
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2024-01-01', '2024-03-20')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(mask_s2_clouds)
dataset = dataset.filterBounds(geometry)
# study area
# geometry = ee.Geometry.Polygon(
#     [[[102.10235701952476, 19.929213464330438],
#       [102.10235701952476, 19.871105510364174],
#       [102.19625579271812, 19.871105510364174],
#       [102.19625579271812, 19.929213464330438]]]
# )


visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

area = dataset.mean()

# Use geemap to display the map
Map = geemap.Map()
Map.setCenter(102.15, 19.90, 12)
Map.addLayer(area.clip(geometry), visualization, 'RGB_2024')



training_sample = table.filterBounds(geometry)
# print(training_sample.getInfo())
# Bands typically used for Sentinel-2 imagery analysis
bands = ['B2', 'B3', 'B4', 'B8']

training = area.select(bands).sampleRegions(
    collection=training_sample,
    properties=['LC'],
    scale=30
)

# print(training.getInfo())

classifier = ee.Classifier.smileCart().train(
    features=training,
    classProperty='LC',
    inputProperties=bands
)

# print(classifier.explain())

classified = area.select(bands).classify(classifier)
Map.addLayer(classified.clip(geometry), 
    {'min': 1, 'max': 3, 'palette': ['blue', 'green', 'red']}, 'classification 2024')

Map.centerObject(training_sample, 11)
# Map.split_map(left_layer='RGB_2024', right_layer='classification 2024')

# To show the map
Map

In [42]:
# get metadata of dataset imageset
print(dataset.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32647', 'crs_transform': [60, 0, 799980, 0, -60, 2300040]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32647', 'crs_transform': [10, 0, 799980, 0, -10, 2300040]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32647', 'crs_transform': [10, 0, 799980, 0, -10, 2300040]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32647', 'crs_transform': [10, 0, 799980, 0, -10, 2300040]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'dimensions': [5490, 5490],

In [47]:
area.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B6',
   'data_type': {'

In [46]:
import pickle
import pandas as pd

# save area to file
with open('area.pkl', 'wb') as f:
    pickle.dump(area, f)

In [48]:
geemap.image_info(area)

AttributeError: module 'geemap.foliumap' has no attribute 'image_info'

In [44]:
print(area.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1,

In [ ]:
ee_object = geemap.shp_to_ee("study_area.shp")

In [23]:
# Add validation samples for 2019
vsample2019 = ee.FeatureCollection('users/banditmienmany/vsample2019')

# it will have additional two nodes
# Cross-validation for 2019 using the classified image of 2019
# node 1
# image to fathure collection
validate2019 = classified.sampleRegions(
    collection=vsample2019,
    properties=['LC'],
    scale=30
)

# Printing can be done directly via Python, but for large data use getInfo() or first().getInfo()
print('Validation 2019:', validate2019.getInfo())

# Calculate the error matrix for 2019
# node 2
error_matrix_2019 = validate2019.errorMatrix('LC', 'classification')

# Display the error matrix and various accuracy metrics
print('Validation error matrix:', error_matrix_2019.getInfo())
print('Overall accuracy:', error_matrix_2019.accuracy().getInfo())
print('Consumer\'s accuracy:', error_matrix_2019.consumersAccuracy().getInfo())
print('Kappa:', error_matrix_2019.kappa().getInfo())
print('Order:', error_matrix_2019.order().getInfo())
print('Producer\'s accuracy:', error_matrix_2019.producersAccuracy().getInfo())


Validation 2019: {'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['classification']}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '00000000000000000000_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000001_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000002_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000003_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000004_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000005_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000000000006_0', 'properties': {'LC': 1, 'classification': 1}}, {'type': 'Feature', 'geometry': None, 'id': '00000000000

it can be a reporter

In [34]:
# get vsample2019 collumn names
print(vsample2019.first().getInfo()["properties"].keys())

dict_keys(['LC'])


In [41]:
# get meata data of the vsample2019
info = vsample2019.first().getInfo()
# visualize json data in a tree structure in html
import json
json_str = json.dumps(info, indent=4)
html = f'<pre>{json_str}</pre>'
# use Ipython to display html
from IPython.display import display, HTML
display(HTML(html))



In [35]:
vsample2019.first().propertyNames().getInfo()

['system:index', 'LC']

In [27]:
type(classified)

ee.image.Image

In [23]:
# Set visualization parameters for the video
video_args = {
    'region': geometry,
    'dimensions': 720,
    'framesPerSecond': 2,
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    'gamma': [0.95, 1.1, 1]  # Adjust gamma for each band if necessary
}


geemap.download_ee_video(dataset, 
    video_args, 
    out_gif='2024.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\xif626\Documents\GitHub\gee_extension\tests\2024.gif


In [16]:


# Define your area of interest
geometry = ee.Geometry.Polygon(
    [[[102.10235701952476, 19.929213464330438],
      [102.10235701952476, 19.871105510364174],
      [102.19625579271812, 19.871105510364174],
      [102.19625579271812, 19.929213464330438]]]
)

# Define the ImageCollection
collection = ee.ImageCollection('MODIS/006/MOD13Q1') \
    .filterDate('2019-01-01', '2019-12-31') \
    .filterBounds(geometry) \
    .select('NDVI')


# Define output video file path
output_video = 'video.gif'  # Specify your output path here

# Use geemap to download the video
geemap.download_ee_video(collection, video_args, output_video)

print(f'Video saved to {output_video}')


Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\xif626\Documents\GitHub\gee_extension\tests\video.gif
Video saved to video.gif


In [43]:
# save geometry to file
# save geometry as shapefile
# Create a feature from the geometry
feature = ee.Feature(geometry)

# Create a feature collection
featureCollection = ee.FeatureCollection([feature])
geemap.ee_to_shp(featureCollection, filename='study_area.shp')

In [4]:
gdf = geemap.ee_to_gdf(training_sample)

In [11]:
# import tif to google earth engine

geemap.image_to_ee("C:\\Users\\xif626\\Downloads\\result.tif")

AttributeError: module 'geemap.foliumap' has no attribute 'image_to_ee'

In [67]:
# save area to file
geemap.ee_export_image(area, filename='area.tif', scale=30, region=geometry)
# save classified to file
geemap.ee_export_image(classified, filename='classified.tif', scale=30, region=geometry)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\xif626\Documents\GitHub\gee_extension\tests\area.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\xif626\Documents\GitHub\gee_extension\tests\classified.tif


In [54]:
viz = {'min': 1, 'max': 3, 'palette': ['blue', 'green', 'red']}
with open('viz.json', 'w') as f:
    json.dump(viz, f)

# Downloading data

In [15]:
import ee
import geemap
# use leaflet
# import geemap.foliumap as geemap

ee.Authenticate()
ee.Initialize(project='gogletetst')

In [21]:
import geopandas as gpd
gdf = gpd.read_file("square.shp")
gdf

CRSError: Invalid projection: epsg:3857: (Internal Proj Error: proj_create: SQLite error on SELECT name, coordinate_system_auth_name, coordinate_system_code, geodetic_crs_auth_name, geodetic_crs_code, conversion_auth_name, conversion_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM projected_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [17]:
# Function to mask clouds using the Sentinel-2 QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)


ee_object = geemap.shp_to_ee("study_area.shp")
geometry = ee_object.geometry()
# Image collection for 2024
dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2024-01-01', '2024-03-20')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(mask_s2_clouds)
dataset = dataset.filterBounds(geometry)


visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

area = dataset.mean()

# Use geemap to display the map
Map = geemap.Map()
Map.setCenter(102.15, 19.90, 12)
Map.addLayer(area.clip(geometry), visualization, 'RGB_2024')


In [18]:
Map

Map(center=[19.9, 102.15], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [10]:
gdf_shichang.explore()

In [2]:
# Create an interactive map.
Map = geemap.Map()

# Load the SkySat Image Collection.
dataset = ee.ImageCollection('SKYSAT/GEN-A/PUBLIC/ORTHO/RGB')

# Select the RGB bands.
rgb = dataset.select(['R', 'G', 'B'])

# Define the visualization parameters.
rgbVis = {
    'min': 11.0,
    'max': 190.0,
}

# Set the map center and zoom level.
Map.setCenter(-70.892, 41.6555, 15)

# Add the layer to the map with the specified visualization parameters.
Map.addLayer(rgb, rgbVis, 'RGB')

# To display the map in a Jupyter notebook
Map

Map(center=[41.6555, -70.892], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

The Planet API Key must be provided.
The Planet API Key must be provided.


In [ ]:
geemap.ee_export_image(image, filename= "test.tif", scale= self.scale, region=image.geometry())